Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.14.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(tenant_id="447af084-8821-49a3-890f-1499b3ea7408", # tenantID
                                    service_principal_id="ca5cebc9-2b50-4548-8f97-b99917157e82", # clientId
                                    service_principal_password="ZlfRN3SvgTFOVule.FXTAI__15WPvfSfC.") # clientSecret

In [3]:
from azureml.core import Workspace

ws = Workspace.get(name="mlWorkspace",
                   auth=sp,
                   subscription_id="539bc2da-eefd-4f46-9b84-6499b0af5dd5")
ws.get_details()

{'id': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourceGroups/mlResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/mlWorkspace',
 'name': 'mlWorkspace',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '43f9f1d1-1489-47c1-8c9a-712b4b9e0973',
 'description': '',
 'friendlyName': 'mlWorkspace',
 'creationTime': '2020-09-24T09:07:46.3558506+00:00',
 'containerRegistry': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourceGroups/mlResourceGroup/providers/Microsoft.ContainerRegistry/registries/43f9f1d1148947c18c9a712b4b9e0973',
 'keyVault': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlresourcegroup/providers/microsoft.keyvault/vaults/mlworkspace8762773565',
 'applicationInsights': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlresourcegroup/providers/microsoft.insights/components/mlworkspace9232446327',
 'identityPrincipalId': '

In [53]:
#ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

mlWorkspace
mlResourceGroup
eastus
539bc2da-eefd-4f46-9b84-6499b0af5dd5


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [5]:
from azureml.train.automl import AutoMLConfig

# task can be one of classification, regression, forecasting
automl_config = AutoMLConfig(task = "classification")

In [6]:
from azureml.core.dataset import Dataset
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = Dataset.Tabular.from_delimited_files(data)

In [7]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automlstep-classification'
project_folder = './automl-pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automlstep-classification,mlWorkspace,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [8]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [9]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [10]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [11]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [12]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [13]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [14]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [15]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [6883696c][2ee1f3d9-92a8-485e-b2b8-0cb6681e3559], (This step will run and generate new outputs)
Submitted PipelineRun ab4c7824-97f7-4e05-bf6f-5706ca4881b9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-classification/runs/ab4c7824-97f7-4e05-bf6f-5706ca4881b9?wsid=/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlResourceGroup/workspaces/mlWorkspace


In [18]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [19]:
pipeline_run.wait_for_completion()

PipelineRunId: ab4c7824-97f7-4e05-bf6f-5706ca4881b9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-classification/runs/ab4c7824-97f7-4e05-bf6f-5706ca4881b9?wsid=/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlResourceGroup/workspaces/mlWorkspace
PipelineRun Status: Running


StepRunId: 0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-classification/runs/0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16?wsid=/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlResourceGroup/workspaces/mlWorkspace
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2020-09-25T07:56:22.350683Z', 'endTimeUtc': '2020-09-25T08:13:42.323472Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'C

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [20]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16/metrics_data, 1 files out of an estimated total of 1


In [21]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_18,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_21,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_35,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_15,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_11,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_9,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_29,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_39,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_1,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_20,...,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_34,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_30,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_16,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_17,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_27,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_5,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_14,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_7,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_25,0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16_31
f1_score_weighted,[0.8634180842598591],[0.7642725876086041],[0.8908391068943605],[0.8628650102993649],[0.8759379039849542],[0.8770717119547734],[0.8963032847961095],[0.8713308742185677],[0.9021127651963996],[0.7579320774824435],...,[0.9084143013294615],[0.8353395018439429],[0.7507000212971882],[0.8828754111160652],[0.8776339493405274],[0.8741194440003212],[0.8353395018439429],[0.8908642194440282],[0.9056577488524223],[0.9081097842584505]
average_precision_score_weighted,[0.9359260261193815],[0.9169180586836007],[0.9362056070511353],[0.9412370340663475],[0.9379312413020887],[0.9380328509830248],[0.9404566949779176],[0.9536822770832154],[0.9505970434373063],[0.9158523773237621],...,[0.9525363906457935],[0.9218741285154002],[0.9224804209002287],[0.9343668075650797],[0.9410461163293812],[0.9170717740414633],[0.920564565868373],[0.9369733513711404],[0.9524128699963818],[0.9536848876131276]
matthews_correlation,[0.4961324404448142],[0.3018466023038423],[0.41787952888447016],[0.28425624568686636],[0.5316287192924629],[0.336275755330834],[0.4873068475173506],[0.3353057207993988],[0.488678780261868],[0.29406957939934797],...,[0.5261182748617416],[0.0],[0.36223653452473004],[0.3704401246257618],[0.34570996806564047],[0.3300724193892132],[0.0],[0.42324132178373874],[0.5151766229945487],[0.5171981369930524]
f1_score_micro,[0.8418816388467375],[0.7128983308042488],[0.9022761760242792],[0.8980273141122913],[0.8576631259484067],[0.8965098634294385],[0.8946889226100152],[0.9019726858877086],[0.9071320182094081],[0.7047040971168437],...,[0.9116843702579667],[0.8880121396054628],[0.6943854324734446],[0.8980273141122913],[0.9001517450682853],[0.8998482549317147],[0.8880121396054628],[0.8986342943854325],[0.90804248861912],[0.9147192716236722]
AUC_micro,[0.8992319719260109],[0.8223775850198374],[0.9711802266274601],[0.9721405265254524],[0.9261375008347129],[0.9677756107220902],[0.9612648032034559],[0.9769440523531999],[0.9781770788959222],[0.8228109449872317],...,[0.9791367340500735],[0.9669456411862366],[0.8424687241670716],[0.9692425871728213],[0.9730858131025764],[0.957392103269542],[0.9621804315638952],[0.9682384446936431],[0.978771624823559],[0.9792922094220101]
accuracy,[0.8418816388467375],[0.7128983308042489],[0.9022761760242792],[0.8980273141122913],[0.8576631259484067],[0.8965098634294385],[0.8946889226100152],[0.9019726858877086],[0.9071320182094081],[0.7047040971168437],...,[0.9116843702579667],[0.8880121396054628],[0.6943854324734446],[0.8980273141122913],[0.9001517450682853],[0.8998482549317147],[0.8880121396054628],[0.8986342943854325],[0.9080424886191198],[0.9147192716236723]
recall_score_weighted,[0.8418816388467375],[0.7128983308042489],[0.9022761760242792],[0.8980273141122913],[0.8576631259484067],[0.8965098634294385],[0.8946889226100152],[0.9019726858877086],[0.9071320182094081],[0.7047040971168437],...,[0.9116843702579667],[0.8880121396054628],[0.6943854324734446],[0.8980273141122913],[0.9001517450682853],[0.8998482549317147],[0.8880121396054628],[0.8986342943854325],[0.9080424886191198],[0.9147192716236723]
precision_score_weighted,[0.9089539713944507],[0.8768359985923969],[0.8877585836712569],[0.8885271361396804],[0.9151819116325929],[0.8

### Retrieve the Best Model

In [22]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16/model_data, 1 files out of an estimated total of 1


In [68]:
type(best_model_output)

azureml.pipeline.core.graph.PortDataReference

In [43]:
best_model_output

Name,Datastore,Path on Datastore,Produced By PipelineRun
best_model_output,workspaceblobstore,azureml/0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16/model_data,ab4c7824-97f7-4e05-bf6f-5706ca4881b9


In [45]:
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [71]:
print(type(best_model_output_name), type(best_model_output))

<class 'str'> <class 'azureml.pipeline.core.graph.PortDataReference'>


In [69]:
best_model_output._path_on_datastore

'azureml/0d2f9e7b-8521-41b4-9bf5-021ec2b4ba16/model_data'

In [26]:
import pickle
import azureml.train.automl

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [70]:
type(best_model)

azureml.automl.runtime.shared.model_wrappers.PipelineWithYTransformations

In [27]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('0',
                                             Pipeline(memory=None,
                                                      steps=[('maxabsscaler',
                                                              MaxAbsScaler(copy=True)),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(boosting_type='gbdt',
                                                          

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [28]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [29]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [35]:
# Visualize the confusion matrix
#pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)
cm

array([[28590,   668],
       [ 1493,  2199]], dtype=int64)

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [36]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,444482a1-fa08-45f7-a40a-1d0a41b96811,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [37]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Performing interactive authentication. Please follow the instructions on the terminal.
WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [38]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [39]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  7f3e473e-76ef-470e-8ab3-e4ec75d1781d


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [40]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [41]:
run_id

'7f3e473e-76ef-470e-8ab3-e4ec75d1781d'

In [42]:
published_pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
pipeline-rest-endpoint,7f3e473e-76ef-470e-8ab3-e4ec75d1781d,azureml.PipelineRun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [50]:
print(pipeline_run.get_file_names())

['logs/azureml/executionlogs.txt', 'logs/azureml/stderrlogs.txt', 'logs/azureml/stdoutlogs.txt']


In [51]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
automlstep-classification,ab4c7824-97f7-4e05-bf6f-5706ca4881b9,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [54]:
ws.get_details()

{'id': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourceGroups/mlResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/mlWorkspace',
 'name': 'mlWorkspace',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '43f9f1d1-1489-47c1-8c9a-712b4b9e0973',
 'description': '',
 'friendlyName': 'mlWorkspace',
 'creationTime': '2020-09-24T09:07:46.3558506+00:00',
 'containerRegistry': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourceGroups/mlResourceGroup/providers/Microsoft.ContainerRegistry/registries/43f9f1d1148947c18c9a712b4b9e0973',
 'keyVault': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlresourcegroup/providers/microsoft.keyvault/vaults/mlworkspace8762773565',
 'applicationInsights': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlresourcegroup/providers/microsoft.insights/components/mlworkspace9232446327',
 'identityPrincipalId': '

In [72]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://93d55bdc-dfc9-4aba-b75c-61c231a3d934.eastus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'QXWjD2h9zCUWTu7TA0AejaEzSlordGUX'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 17,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 971,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
          {
            "age": 87,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 471,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
      ]
    }

In [73]:
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

In [74]:
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

In [75]:
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": ["no", "no"]}


In [76]:
 from azureml.core.webservice import AciWebservice
 aciws = AciWebservice(ws, 'deploy-best-model')

In [77]:
aciws.update(enable_app_insights=True)

In [78]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
#ws = Workspace.from_config()

# Set with the deployment name
name = "deploy-best-model"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2020-09-25T13:35:34,973644246+00:00 - rsyslog/run 
2020-09-25T13:35:34,977929540+00:00 - gunicorn/run 
2020-09-25T13:35:34,982785834+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_a20ffb9f71fda8a712772ab1c6b1d845/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a20ffb9f71fda8a712772ab1c6b1d845/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a20ffb9f71fda8a712772ab1c6b1d845/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a20ffb9f71fda8a712772ab1c6b1d845/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a20ffb9f71fda8a712772ab1c6b1d845/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-09-25T13:35:34,997089615+00:00 - nginx/run 
rsyslogd